In [4]:
import os
os.getcwd()

'/Users/isenb047/Documents/INSOFE-PGP/GEM1.0'

In [1]:
import numpy as np
import statistics as stat
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.model_selection import train_test_split
import pandas as pd
import random

def split(data):
    Y=data["Target"]
    X=data.copy()
    #X.drop(["TenYearCHD",axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)
    return X_train,X_test,y_train,y_test
    

In [ ]:
def model_building(X_train,y_train,X_test,y_test,model):
        model.fit(X_train,y_train)
        trainpreds=model.predict(X_train)
        acc=accuracy_score(y_train,trainpreds)
        rec=recall_score(y_train,trainpreds,pos_label=1)

        testpreds=model.predict(X_test)
        testacc=accuracy_score(y_test,testpreds)
        testrec=recall_score(y_test,testpreds,pos_label=1)
        m=pd.DataFrame({'A':[acc,rec,testacc,testrec]})    
        return(m)

    return Exp3_metric

In [ ]:
def model_voting(DS,X_test2):
    
    
    DS_preds=pd.DataFrame({"ID": DS["ID"]})
    DS_testpreds=pd.DataFrame({"ID":X_test2["ID"]})
    
    X_test2.drop(["ID"],axis=1,inplace=True)
    
    X_train=DS.copy()
    y_train=DS.Target
    X_train.drop(["ID","Target"],axis=1,inplace=True)
    
    
 
    model1 = DecisionTreeClassifier()
    
    model1.fit(X_train,y_train)
    DS_preds['DT']=model1.predict(X_train)
    DS_testpreds['DT']=model1.predict(X_test2)
    #print(DS_testpreds.shape)
    
    model2 = KNeighborsClassifier()
    model2.fit(X_train,y_train)
    DS_preds['KNN']=model2.predict(X_train)
    DS_testpreds['KNN']=model2.predict(X_test2)
    
    
    model3= LogisticRegression()
    model3.fit(X_train,y_train)
    DS_preds['Logistic']=model3.predict(X_train)
    DS_testpreds['Logistic']=model3.predict(X_test2)
    
    return DS_preds,DS_testpreds

def Exp1(X_train, X_test, y_train, y_test,name):
    
    metrics=model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier())
    metrics['Logistic']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,LogisticRegression()))
    metrics['KNN']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,KNeighborsClassifier()))
    metrics['RandomForest']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier()))
    metrics['RandomForest_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier(class_weight='balanced')))
    metrics['DTC_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier(class_weight='balanced')))
    metrics['AdaBoost']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,AdaBoostClassifier()))
    #metrics['XgBoost']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,AdaBoostClassifier()))
    
    metrics.rename(columns={'A':'DecsionTree'},inplace=True)
    metrics['metric']=['trainaccuracy','testaccuracy','trainrecall','testrecall']
    #metrics.set_index('metric')
    metrics.to_csv("Exp1_metrics"+name+".csv")
    print('Experimet1results written to Exp1_metrics.csv')
    return metrics

In [ ]:
    
## function divides train data majority and minority samples; then divide majority
def prepareData(X_train):
    #Y=data["Target"]
    #X=data.copy()
    #X.drop(["TenYearCHD",axis=1,inplace=True)
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)
    
    
    data1=X_train.copy()
    data_Majority=data1[data1.Target==0]
    data_Minority=data1[data1.Target==1]
    
    s = list(range(data_Majority.shape[0]))
    s1=np.ceil(data_Minority.shape[0]+data_Minority.shape[0]*0.40).astype('int')
    print(s1)
    random.shuffle(s)
    
    first_sample = s[-s1:]
    del s[-s1:]
    second_sample = s[-s1:]
    del s[-s1:]
    third_sample=s[-s1:]
    
    data_Majority_ds1=data_Majority.iloc[first_sample,:]
    data_Majority_ds2=data_Majority.iloc[second_sample,:]
    data_Majority_ds3=data_Majority.iloc[third_sample,:]
    
    print(data_Majority_ds1.shape)
    print(data_Majority_ds2.shape)
    print(data_Majority_ds3.shape)
    
    DS1=data_Majority_ds1.append(data_Minority)
    DS2=data_Majority_ds2.append(data_Minority)
    DS3=data_Majority_ds3.append(data_Minority)
    print(DS1.shape)
    print(DS2.shape)
    print(DS3.shape)
    print("Datasets prepared and retured; pass it to Exp2 and Exp3 get the results")
    return DS1,DS2,DS3


def Exp2(DS1,DS2,DS3,X_test,y_test,name):
    X_train=DS1.copy()
    y_train=DS1.Target
    print("entered")
    metrics_exp2=model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier())
    metrics_exp2['DS1_Logistic']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,LogisticRegression()))
    metrics_exp2['DS1_KNN']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,KNeighborsClassifier()))
    metrics_exp2['DS1_RandomForest']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier()))
    metrics_exp2['DS1_RandomForest_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier(class_weight='balanced')))
    metrics_exp2['DS1_DTC_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier(class_weight='balanced')))
    metrics_exp2['DS1_AdaBoost']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,AdaBoostClassifier()))
    
    metrics_exp2.rename(columns={'A':'DS1_DTC'},inplace=True)
    
    X_train=DS2.copy()
    y_train=DS2.Target
    metrics_exp2['DS2_DecisionTree']=model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier())
    metrics_exp2['DS2_Logistic']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,LogisticRegression()))
    metrics_exp2['DS2_KNN']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,KNeighborsClassifier()))
    metrics_exp2['DS2_RandomForest']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier()))
    metrics_exp2['DS2_RandomForest_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier(class_weight='balanced')))
    metrics_exp2['DS2_DTC_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier(class_weight='balanced')))
    metrics_exp2['DS2_AdaBoost']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,AdaBoostClassifier()))
 
    X_train=DS3.copy()
    y_train=DS3.Target
    metrics_exp2['DS3_DecisionTree']=model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier())
    metrics_exp2['DS3_Logistic']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,LogisticRegression()))
    metrics_exp2['DS3_KNN']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,KNeighborsClassifier()))
    metrics_exp2['DS3_RandomForest']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier()))
    metrics_exp2['DS3_RandomForest_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,RandomForestClassifier(class_weight='balanced')))
    metrics_exp2['DS3_DTC_classimabalnce']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,DecisionTreeClassifier(class_weight='balanced')))
    metrics_exp2['DS3_AdaBoost']=(model_building(X_train.drop(['ID','Target'],axis=1).copy(),y_train,X_test.drop(['ID','Target'],axis=1).copy(),y_test,AdaBoostClassifier()))

    print("exit")
    metrics_exp2['metric']=['trainaccuracy','testaccuracy','trainrecall','testrecall']
    #metrics_exp2.set_index('metric',inplace=True)
    #metrics_exp2.head(10)
    metrics_exp2.to_csv("metrics_exp2"+name+".csv")
    print("metrics_exp2 has written to csv metrics_exp2")
    return metrics_exp2
    
def Exp3(DS1,DS2,DS3,X_test,y_test,name):
    
    DS1_train_pred,DS1_test_pred=model_voting(DS1,X_test.drop('Target',axis=1).copy())
    DS2_train_pred,DS2_test_pred=model_voting(DS2,X_test.drop('Target',axis=1).copy())
    DS3_train_pred,DS3_test_pred=model_voting(DS3,X_test.drop('Target',axis=1).copy())
    
    FinalTest_pred=pd.merge(pd.merge(DS1_test_pred,DS2_test_pred,on='ID',how='inner'),DS3_test_pred,on='ID',how='inner')
    FinalTest_pred['MODE']=FinalTest_pred.drop('ID',axis=1).mode(axis=1)[0]
    
    test_acc=accuracy_score(y_test,FinalTest_pred['MODE'])
    test_recall=recall_score(y_test,FinalTest_pred['MODE'],pos_label=1)
    
    
    DS1_train_pred['Target']=DS1['Target']
    DS2_train_pred['Target']=DS2['Target']
    DS3_train_pred['Target']=DS3['Target']
    
    
    DS1_train_0=DS1_train_pred[DS1_train_pred.Target==0]
    DS1_train_1=DS1_train_pred[DS1_train_pred.Target==1]
    DS2_train_0=DS2_train_pred[DS2_train_pred.Target==0]
    DS2_train_1=DS2_train_pred[DS2_train_pred.Target==1]
    DS3_train_0=DS3_train_pred[DS3_train_pred.Target==0]
    DS3_train_1=DS3_train_pred[DS3_train_pred.Target==1]
    
    
    train_global_0=DS1_train_0.copy()
    train_global_0=train_global_0.append(DS2_train_0).append(DS3_train_0)
    #train_global_0=train_global_0.append(DS3_train_0)
    print(train_global_0.shape)
    
    
    train_global_1=DS1_train_1.copy()
    train_global_1=pd.merge(pd.merge(train_global_1,DS2_train_1,how='inner',on='ID'),DS3_train_1,on='ID',how='inner')
    #train_global_1=pd.merge(train_global_1,DS3_train_1,how='inner',on='ID')
    train_global_1.columns
    
    train_global_1.drop(['Target_x','Target_y'],axis=1,inplace=True)
    
    
    train_global_1_mode=pd.DataFrame({'ID':train_global_1.ID})
    train_global_1_mode['DT']=train_global_1[['DT','DT_x','DT_y']].mode(axis=1)[0]
    train_global_1_mode['KNN']=train_global_1[['KNN','KNN_x','KNN_y']].mode(axis=1)[0]
    train_global_1_mode['Logistic']=train_global_1[['Logistic','Logistic_x','Logistic_y']].mode(axis=1)[0]
    train_global_1_mode['Target']=train_global_1[['Target']]
    
    train_global=train_global_0.copy()
    train_global=train_global.append(train_global_1_mode)
    
    
    y_pred=train_global[['DT','KNN','Logistic']].mode(axis=1)[0]
    
    
    train_acc=accuracy_score(train_global.Target,y_pred)
    train_recall=recall_score(train_global.Target,y_pred,pos_label=1)

    m=[train_acc,train_recall,test_acc,test_recall]
    Exp3_metric=pd.DataFrame({'Metric':['trainaccuracy','trainrecall','testaccuracy','testrecall'],name:m})
    Exp3_metric.to_csv("Exp3_metric"+name+".csv")
    print("metrics file written: Exp3_metric.csv ")